In [9]:
import sys

sys.path.append("../")

%load_ext autoreload
%autoreload 2

import random

import numpy as np
import pandas as pd
from simulator.model.robust_mse import RobustBidMSE
from simulator.model.simple import SimpleBid
from simulator.simulation.utils_visualization import plot_metric_with_error_CTR
from simulator.validation.check_results import autobidder_check
from tqdm import tqdm

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# This notebook provides a guideline how to simulate and check Robust PID Bidder for all the campaigns in the dataset

In [10]:
loss_type = 'MSE'

# Load data

In [11]:
auction_mode = 'FPA'

campaigns_path = "../data/subsample_campaigns.csv"
stats_path = "../data/subsample_stats.csv"

In [12]:
campaign_df = pd.read_csv(campaigns_path)
stats_df = pd.read_csv(stats_path)

In [13]:
campaign_df.describe()

,campaign_id,loc_id,item_id,campaign_start,campaign_end,auction_budget,microcat_ext,logical_category,region_id
count,7.000000e+01,70.000000,7.000000e+01,7.000000e+01,7.000000e+01,70.000000,7.000000e+01,70.000000,70.000000
mean,7.695222e+07,644788.385714,3.515224e+09,5.298478e+08,5.301514e+08,1238.622143,7.844751e+05,2.769286,644358.000000
std,4.957660e+06,10712.608247,8.250976e+08,3.126187e+05,3.845838e+05,1906.618616,1.013575e+06,1.485689,10649.319309
min,5.986990e+07,621630.000000,1.073987e+09,5.295283e+08,5.297184e+08,75.840000,2.400000e+01,1.000000,621590.000000
25%,7.307796e+07,637672.500000,2.998627e+09,5.295718e+08,5.298089e+08,228.960000,3.779750e+03,1.380000,637650.000000
50%,7.567955e+07,646410.000000,3.915603e+09,5.297500e+08,5.299637e+08,547.200000,5.533500e+03,2.350000,645790.000000
75%,8.077054e+07,653240.000000,4.063275e+09,5.301044e+08,5.305207e+08,1934.687500,2.096862e+06,3.410000,653240.000000
max,8.776145e+07,661950.000000,4.577216e+09,5.305651e+08,5.308243e+08,13829.760000,2.278193e+06,5.370000,661460.000000


## Robust LP Bid vs LP Bid

In [14]:
# """
# Точная реализация генерации шума по Wang & Dong (2023)
# на основе формул из статьи с энтропией и логарифмами

# Из статьи Wang, X., & Dong, H. (2023):
# - Формула (15): H(y*|x*,D) = MI(y*;ω|x*,D) + E(H(y*|x*,ω))
# - Формула (17): Ĥ(y*|x*,D) = -∑_c (1/T ∑_t p(y=c|x*,ω̂_t) log(1/T ∑_t p(y=c|x*,ω̂_t)))
# - Формула (18): E(Ĥ(y*|x*,ω)) = -1/T ∑_{t,c} p(y=c|x*,ω̂_t) log p(y=c|x*,ω̂_t)
# - Формула (19): M̂I(y*;ω|x*,D) = Ĥ(y*|x*,D) - E(Ĥ(y*|x*,ω))
# """

# import numpy as np
# import pandas as pd
# from scipy import stats
# import matplotlib.pyplot as plt

# class WangDongCTRNoise:
#     """
#     Генератор шума для CTR на основе точных формул из Wang & Dong (2023)
#     """

#     def __init__(self, random_state=1337):
#         np.random.seed(random_state)
#         self.random_state = random_state

#     def monte_carlo_dropout_predictions(self, ctr_predicted, T=50, dropout_rate=0.05):
#         """
#         Генерация T предсказаний через Monte Carlo Dropout
#         Соответствует процедуре из статьи для получения ω̂_t

#         Parameters:
#         -----------
#         ctr_predicted : array-like
#             Базовые предсказания CTR (p(y=1|x*,ω))
#         T : int
#             Количество MC семплов (как в статье)
#         dropout_rate : float
#             Dropout rate (рекомендуется 0.05 из экспериментов статьи)

#         Returns:
#         --------
#         mc_predictions : np.array shape (T, N)
#             T предсказаний для каждого из N семплов
#         """
#         n_samples = len(ctr_predicted)
#         ctr_clipped = np.clip(ctr_predicted, 1e-7, 1-1e-7)  # Избегаем log(0)

#         mc_predictions = []

#         for t in range(T):
#             # Симулируем dropout через модификацию логитов
#             # Это приближение к реальному MC Dropout из статьи
#             logits = np.log(ctr_clipped / (1 - ctr_clipped))

#             # Dropout mask - случайно обнуляем некоторые "нейроны"
#             dropout_mask = np.random.binomial(1, 1-dropout_rate, n_samples)

#             # Добавляем шум, пропорциональный неопределенности
#             # Больше шума там, где модель менее уверена (средние значения CTR)
#             uncertainty = 4 * ctr_clipped * (1 - ctr_clipped)  # Максимум при CTR=0.5
#             noise_std = dropout_rate * uncertainty

#             logit_noise = np.random.normal(0, noise_std)
#             noisy_logits = logits + logit_noise * dropout_mask

#             # Обратное преобразование в вероятности
#             mc_ctr = 1 / (1 + np.exp(-noisy_logits))
#             mc_predictions.append(mc_ctr)

#         return np.array(mc_predictions)

#     def calculate_predictive_entropy(self, mc_predictions):
#         """
#         Вычисление предсказательной энтропии Ĥ(y*|x*,D)
#         Формула (17) из статьи

#         Parameters:
#         -----------
#         mc_predictions : np.array shape (T, N)
#             MC предсказания

#         Returns:
#         --------
#         predictive_entropy : np.array shape (N,)
#             Предсказательная энтропия для каждого семпла
#         """
#         T, N = mc_predictions.shape

#         # Усредненные предсказания: 1/T ∑_t p(y=c|x*,ω̂_t)
#         avg_p1 = np.mean(mc_predictions, axis=0)  # P(y=1)
#         avg_p0 = 1 - avg_p1  # P(y=0)

#         # Избегаем log(0)
#         avg_p1 = np.clip(avg_p1, 1e-7, 1-1e-7)
#         avg_p0 = np.clip(avg_p0, 1e-7, 1-1e-7)

#         # Формула (17): Ĥ(y*|x*,D) = -∑_c (avg_p_c * log(avg_p_c))
#         predictive_entropy = -(avg_p1 * np.log(avg_p1) + avg_p0 * np.log(avg_p0))

#         return predictive_entropy

#     def calculate_expected_entropy(self, mc_predictions):
#         """
#         Вычисление ожидаемой энтропии E(Ĥ(y*|x*,ω))
#         Формула (18) из статьи

#         Parameters:
#         -----------
#         mc_predictions : np.array shape (T, N)
#             MC предсказания

#         Returns:
#         --------
#         expected_entropy : np.array shape (N,)
#             Ожидаемая энтропия для каждого семпла
#         """
#         T, N = mc_predictions.shape

#         # Избегаем log(0)
#         mc_predictions_safe = np.clip(mc_predictions, 1e-7, 1-1e-7)
#         mc_predictions_0 = 1 - mc_predictions_safe

#         # Формула (18): E(Ĥ(y*|x*,ω)) = -1/T ∑_{t,c} p(y=c|x*,ω̂_t) log p(y=c|x*,ω̂_t)
#         entropy_per_sample = -(mc_predictions_safe * np.log(mc_predictions_safe) +
#                               mc_predictions_0 * np.log(mc_predictions_0))

#         expected_entropy = np.mean(entropy_per_sample, axis=0)

#         return expected_entropy

#     def calculate_mutual_information(self, predictive_entropy, expected_entropy):
#         """
#         Вычисление взаимной информации M̂I(y*;ω|x*,D)
#         Формула (19) из статьи

#         Parameters:
#         -----------
#         predictive_entropy : np.array
#             Предсказательная энтропия
#         expected_entropy : np.array
#             Ожидаемая энтропия

#         Returns:
#         --------
#         mutual_information : np.array
#             Взаимная информация (epistemic uncertainty)
#         """
#         # Формула (19): M̂I(y*;ω|x*,D) = Ĥ(y*|x*,D) - E(Ĥ(y*|x*,ω))
#         mutual_information = predictive_entropy - expected_entropy

#         # MI не может быть отрицательной (ограничиваем снизу)
#         mutual_information = np.maximum(mutual_information, 0)

#         return mutual_information

#     def generate_wang_dong_noise(self, ctr_predicted, T=50, dropout_rate=0.05,
#                                 noise_type='total', scale_factor=1.0):
#         """
#         Генерация шума по точной методологии Wang & Dong (2023)

#         Parameters:
#         -----------
#         ctr_predicted : array-like
#             Предсказанные значения CTR
#         T : int
#             Количество MC семплов (из статьи: обычно 50)
#         dropout_rate : float
#             Dropout rate (из статьи: обычно 0.05)
#         noise_type : str
#             'epistemic' - только epistemic uncertainty (MI)
#             'aleatoric' - только aleatoric uncertainty (expected entropy)
#             'total' - общая uncertainty (predictive entropy)
#         scale_factor : float
#             Масштабирующий коэффициент для шума

#         Returns:
#         --------
#         noise : np.array
#             Сгенерированный шум
#         uncertainty_info : dict
#             Детальная информация об uncertainty
#         """
#         # Шаг 1: Генерируем MC предсказания (симуляция dropout)
#         mc_predictions = self.monte_carlo_dropout_predictions(
#             ctr_predicted, T=T, dropout_rate=dropout_rate
#         )

#         # Шаг 2: Вычисляем три типа uncertainty по формулам из статьи
#         predictive_entropy = self.calculate_predictive_entropy(mc_predictions)
#         expected_entropy = self.calculate_expected_entropy(mc_predictions)
#         mutual_information = self.calculate_mutual_information(
#             predictive_entropy, expected_entropy
#         )

#         # Шаг 3: Выбираем тип uncertainty для генерации шума
#         if noise_type == 'epistemic':
#             # Epistemic uncertainty = MI (неопределенность модели)
#             uncertainty_values = mutual_information
#         elif noise_type == 'aleatoric':
#             # Aleatoric uncertainty = Expected entropy (неопределенность данных)
#             uncertainty_values = expected_entropy
#         elif noise_type == 'total':
#             # Total uncertainty = Predictive entropy
#             uncertainty_values = predictive_entropy
#         else:
#             raise ValueError(f"Unknown noise_type: {noise_type}")

#         # Шаг 4: Генерируем шум с дисперсией, пропорциональной uncertainty
#         # Чем больше uncertainty, тем больше потенциальный шум
#         noise_std = scale_factor * np.sqrt(uncertainty_values)
#         noise = np.random.normal(0, noise_std)

#         # Информация для анализа
#         uncertainty_info = {
#             'predictive_entropy': predictive_entropy,
#             'expected_entropy': expected_entropy,
#             'mutual_information': mutual_information,
#             'noise_std': noise_std,
#             'mc_predictions_std': np.std(mc_predictions, axis=0),
#             'method_params': {
#                 'T': T,
#                 'dropout_rate': dropout_rate,
#                 'noise_type': noise_type,
#                 'scale_factor': scale_factor
#             }
#         }

#         return noise, uncertainty_info

#     def suggest_scale_factor(self, ctr_predicted, target_noise_std=0.05):
#         """
#         Автоматический подбор scale_factor для достижения желаемого уровня шума

#         Parameters:
#         -----------
#         ctr_predicted : array-like
#             Предсказанные значения CTR
#         target_noise_std : float
#             Желаемое стандартное отклонение шума

#         Returns:
#         --------
#         optimal_scale_factor : float
#             Рекомендуемый scale_factor
#         """
#         # Тестируем разные scale_factor
#         test_scales = np.logspace(-2, 1, 20)  # от 0.01 до 10

#         best_scale = 1.0
#         min_diff = float('inf')

#         for scale in test_scales:
#             noise, _ = self.generate_wang_dong_noise(
#                 ctr_predicted, scale_factor=scale, T=20  # Быстрая оценка
#             )
#             actual_std = np.std(noise)
#             diff = abs(actual_std - target_noise_std)

#             if diff < min_diff:
#                 min_diff = diff
#                 best_scale = scale

#         return best_scale

# # Основная функция для использования
# def generate_wang_dong_2023_noise(ctr_values, target_noise_std=0.05,
#                                  noise_type='total', auto_scale=True):
#     """
#     Простая функция для генерации шума по Wang & Dong (2023)

#     Parameters:
#     -----------
#     ctr_values : array-like
#         Ваши предсказанные CTR значения
#     target_noise_std : float
#         Желаемое стандартное отклонение шума
#     noise_type : str
#         'epistemic', 'aleatoric', или 'total'
#     auto_scale : bool
#         Автоматически подобрать scale_factor

#     Returns:
#     --------
#     noise : np.array
#         Сгенерированный шум
#     citation_info : str
#         Готовый текст для цитирования в статье
#     """
#     generator = WangDongCTRNoise(random_state=42)

#     if auto_scale:
#         scale_factor = generator.suggest_scale_factor(ctr_values, target_noise_std)
#         # print(f"Автоматически подобранный scale_factor: {scale_factor:.3f}")
#     else:
#         scale_factor = 1.0

#     noise, info = generator.generate_wang_dong_noise(
#         ctr_values,
#         noise_type=noise_type,
#         scale_factor=scale_factor,
#         T=50,  # Как в статье
#         dropout_rate=0.05  # Оптимальное значение из статьи
#     )

#     return noise, _ # citation_info.strip()

# # Пример использования
# if __name__ == "__main__":
#     # Тестовые данные - ваши CTR значения
#     np.random.seed(42)
#     test_ctr = np.random.uniform(0.01, 0.3, 1000)

#     print("=== Генерация шума по Wang & Dong (2023) ===")
#     noise, citation = generate_wang_dong_2023_noise(
#         test_ctr,
#         target_noise_std=0.05,
#         noise_type='total'
#     )

#     print("\n" + "="*50)
#     print("ТЕКСТ ДЛЯ СТАТЬИ:")
#     print("="*50)
#     print(citation)

#     # Валидация
#     generator = WangDongCTRNoise()
#     # info = generator.validate_methodology(test_ctr, plot=False)

In [7]:
def create_noised_stats_mse(stats_df, stats_save_path, old_ctr, eps, auction_mode, seed):
    random.seed(seed)
    np.random.seed(seed)

    grouped = stats_df.groupby('campaign_id')
    for campaign_id, group in grouped:
        old_ctr = group['CTRPredicts'].values

        noise, citation = generate_wang_dong_2023_noise(
            old_ctr,
            target_noise_std=eps,
            noise_type='total'
        )

        # noise = np.random.rand(old_ctr.size)
        # noise = noise / np.linalg.norm(noise) * np.sqrt(2*eps)
        stats_df.loc[stats_df.campaign_id == campaign_id, 'CTRPredicts_noised'] = np.clip(old_ctr + noise, 0.01, 0.1)

    stats_df.to_csv(stats_save_path)


In [15]:
eps_set_data = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 1e-12]
# eps_set_model = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 1e-12]

bidder_types = ['simple', 'robust']
seeds = list(range(10))

metrics_list = []

old_ctr = np.array(stats_df.CTRPredicts.copy())

stats_save_path = f"../data/data/{auction_mode.lower()}/stats_{auction_mode.lower()}_filtered_train_noised.csv"

for eps in tqdm(eps_set_data):
    # for factor in [0.1, 1., 10, 100]:
    # eps_data = eps_model # * factor
    for seed in seeds:
        # print(seed)
        # Add noise to data
        create_noised_stats_mse(stats_df, stats_save_path, old_ctr, eps, auction_mode, seed)

        cpc = 300.

        # Simple bid
        res_simple = autobidder_check(
            bidder=SimpleBid,
            params={"input_campaigns": campaigns_path,
                    "input_stats": stats_save_path,
                    'eps': eps,
                    'p': 1,
                    'q': 1,
                    'LP': True,
                    'CPC': cpc},
            loss_type=loss_type
        )
        metrics_list.append({
            'eps': eps,
            # 'eps_model': eps,
            'bidder_type': 'simple',
            'seed': seed,
            'tvc': res_simple['score'][0],
            'cpc_percent': res_simple['score'][1],
            'cpc_avg': res_simple['score'][2]
        })

        # Robust bid
        res_robust = autobidder_check(
            bidder=RobustBidMSE,
            params={"input_campaigns": campaigns_path,
                    "input_stats": stats_save_path,
                    'eps': eps,
                    'gamma': 1.,
                    'beta': 1.,
                    'lambda_': 1.,
                    'chi': 1.,
                    'theta': 1.,
                    'delta': 1.,
                    'kappa': 1.,
                    'LP': True,
                    'CPC': cpc},
            loss_type=loss_type
        )

        metrics_list.append({
            'eps': eps,
            # 'eps_model': eps_model,
            'bidder_type': 'robust',
            'seed': seed,
            'tvc': res_robust['score'][0],
            'cpc_percent': res_robust['score'][1],
            'cpc_avg': res_robust['score'][2]
        })

metrics_df = pd.DataFrame(metrics_list, columns=['eps', 'bidder_type', 'seed', 'tvc', 'cpc_percent', 'cpc_avg'])

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [15:47<00:00, 135.36s/it]


In [16]:
agg_metrics = metrics_df.groupby(['eps', 'bidder_type']).agg(
    mean_tvc=('tvc', 'mean'),
    std_tvc=('tvc', 'std'),
    mean_cpc_percent=('cpc_percent', 'mean'),
    std_cpc_percent=('cpc_percent', 'std'),
    mean_cpc_avg=('cpc_avg', 'mean'),
    std_cpc_avg=('cpc_avg', 'std')
).reset_index()

agg_metrics

,eps,bidder_type,mean_tvc,std_tvc,mean_cpc_percent,std_cpc_percent,mean_cpc_avg,std_cpc_avg
0,1.000000e-12,robust,2.260368,0.0,0.0,0.0,468.064523,0.0
1,1.000000e-12,simple,2.260151,0.0,0.0,0.0,468.108536,0.0
2,5.000000e-04,robust,2.787040,0.0,0.0,0.0,331.573948,0.0
3,5.000000e-04,simple,2.260151,0.0,0.0,0.0,468.108536,0.0
4,1.000000e-03,robust,3.072124,0.0,0.0,0.0,312.071788,0.0
5,1.000000e-03,simple,2.260151,0.0,0.0,0.0,468.108536,0.0
6,5.000000e-03,robust,3.407976,0.0,0.0,0.0,291.231927,0.0
7,5.000000e-03,simple,2.251925,0.0,0.0,0.0,469.767519,0.0
8,1.000000e-02,robust,3.546123,0.0,0.0,0.0,282.363859,0.0
9,1.000000e-02,simple,2.172647,0.0,0.0,0.0,474.291485,0.0


In [17]:
agg_metrics.to_csv(f'../results/metrics_{loss_type.lower()}_BAT_CTR.csv')

### TVC

In [ ]:
mean_tvc_simple_ctr = np.array(agg_metrics[agg_metrics.bidder_type == 'simple'].mean_tvc)
mean_tvc_robust_ctr = np.array(agg_metrics[agg_metrics.bidder_type == 'robust'].mean_tvc)

# results from another notebook with normal noise
mean_tvc_simple_norm = np.array([2.26495932, 2.26782824, 2.19687101, 2.08576958, 2.05811226,
       1.93404816, 1.89193563])
mean_tvc_robust_norm = np.array([2.26483722, 2.79915243, 3.03667371, 3.34679585, 3.48146273,
       3.91136865, 4.0522365 ])

In [18]:
(mean_tvc_robust_ctr / mean_tvc_simple_ctr), (mean_tvc_robust_norm / mean_tvc_simple_norm)

(array([1.00009584, 1.23312083, 1.35925602, 1.51516363, 1.63840515,
        1.94609533, 1.93573035]),
 array([0.99994609, 1.23428767, 1.38227219, 1.60458561, 1.69158058,
        2.02237397, 2.14184692]))

In [17]:
(mean_tvc_robust_ctr / mean_tvc_simple_ctr).mean(), (mean_tvc_robust_norm / mean_tvc_simple_norm).mean()

(1.5182667368113836, 1.5824132896383467)

### CPC

In [ ]:
mean_cpc_simple_ctr = np.array(agg_metrics[agg_metrics.bidder_type == 'simple'].mean_cpc_avg)
mean_cpc_robust_ctr = np.array(agg_metrics[agg_metrics.bidder_type == 'robust'].mean_cpc_avg)

# results from another notebook with normal noise
mean_cpc_simple_norm = np.array([515.13199505, 514.81091864, 518.49015618, 523.06691103,
       524.31452656, 525.98242848, 525.52466516])
mean_cpc_robust_norm = np.array([515.05495171, 361.7245655 , 339.32337875, 316.72874209,
       306.37696331, 273.53070906, 256.42070281])

In [21]:
(mean_cpc_robust_ctr / mean_cpc_simple_ctr), (mean_cpc_robust_norm / mean_cpc_simple_norm)

(array([0.99989679, 0.69479282, 0.65160455, 0.60698901, 0.57981732,
        0.49342042, 0.46011147]),
 array([0.99985044, 0.70263577, 0.65444517, 0.60552242, 0.58433812,
        0.52003773, 0.48793276]))

In [20]:
(mean_cpc_robust_ctr / mean_cpc_simple_ctr).mean(), (mean_cpc_robust_norm / mean_cpc_simple_norm).mean()

(0.6409474830510654, 0.6506803447485854)

In [ ]:
plot_metric_with_error_CTR(
    eps_set=eps_set,
    agg_metrics=agg_metrics,
    metric_mean_col='mean_tvc',
    metric_std_col='std_tvc',
    metric_name='TVC',
    y_label='Total Value Clicks',
    loss_type=loss_type
)

plot_metric_with_error_CTR(
    eps_set=eps_set,
    agg_metrics=agg_metrics,
    metric_mean_col='mean_cpc_percent',
    metric_std_col='std_cpc_percent',
    metric_name='CPC Percent',
    y_label='Cost per Click (%)',
    loss_type=loss_type
)

plot_metric_with_error_CTR(
    eps_set=eps_set,
    agg_metrics=agg_metrics,
    metric_mean_col='mean_cpc_avg',
    metric_std_col='std_cpc_avg',
    metric_name='CPC Avg',
    y_label='Average Cost per Click',
    loss_type=loss_type
)
